# Meta-Learning 测试 Notebook

本notebook用于测试Meta-Learning方法，训练RNN成为"快速学习器"，
使其能够从1D规则快速适应并泛化到2D组合任务。

## 核心思想
- **Meta-Training**：训练RNN学习"如何快速学习"
  - Support Set: 所有1-level difference的1D规则样本（相邻pair）
  - Query Set: 2D规则样本（需要组合两个维度）
- **Meta-Testing**：测试1D→2D的泛化能力

## 🚀 Colab使用说明
1. **启用GPU**: Runtime -> Change runtime type -> GPU
2. **上传项目**: 将项目文件上传到Colab
3. **运行所有Cells**: 按顺序运行所有代码块

## 1. 环境设置

In [ ]:
# 修复Colab中的NumPy兼容性问题
import sys
import subprocess
import os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 修复NumPy版本兼容性...")
    try:
        import numpy as np
        if np.__version__.startswith('2.'):
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--force-reinstall", "numpy==1.26.4"])
            if 'numpy' in sys.modules:
                del sys.modules['numpy']
            import numpy as np
            print(f"✓ NumPy已降级到版本: {np.__version__}")
    except:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numpy==1.26.4"])
        import numpy as np
    
    # 安装其他依赖
    for pkg in ['matplotlib', 'tqdm']:
        try:
            __import__(pkg)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
    
    project_root = '/content'
    if os.path.exists('/content') and 'wine-spave' in os.listdir('/content'):
        project_root = '/content/wine-spave'
        os.chdir(project_root)
else:
    # 本地环境
    current_dir = os.getcwd()
    if 'notebooks' in current_dir:
        project_root = os.path.dirname(current_dir)
    else:
        project_root = current_dir
    
    # 检查依赖
    for pkg in ['numpy', 'torch', 'matplotlib', 'tqdm']:
        try:
            __import__(pkg)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 添加项目路径
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"项目根目录: {project_root}")

# GPU检测
if torch.cuda.is_available():
    device = "cuda"
    print(f"✓ GPU可用: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    print("⚠ GPU不可用，将使用CPU")

# 导入项目模块
from data import GridDataGenerator
from models import get_model
from meta_learning import (
    meta_train, 
    meta_test_simple,
    create_meta_learning_args
)

print("✓ 所有模块导入完成")

## 2. 参数设置

In [ ]:
# 基础参数类
class Args:
    def __init__(self):
        # 设备设置
        self.use_cuda = torch.cuda.is_available()
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.seed = 42
        
        # 数据集设置
        self.use_images = False
        self.image_dir = 'images/faces16'
        self.training_regime = 'ungrouped'
        self.grid_size = 4
        self.ctx_order = 'first'
        self.inner_4x4 = False
        
        # 模型设置
        self.model_name = 'rnn'
        self.ctx_scale = 1.0
        self.measure_grad_norm = False
        
        # Meta-Learning参数
        self.meta_lr = 0.001
        # Support Set: 使用所有1-level difference的样本（rank diff = 1）
        # Query Set: 使用所有2D规则样本（cong == -1）
        self.n_support = 64  # 参数保留但实际使用所有1-level difference样本
        self.n_query = 64    # 2D查询样本数
        self.n_meta_iterations = 500  # 可以调整为≥10000用于正式训练
        self.n_tasks_per_batch = 4

# 创建参数对象
args = Args()

# 设置随机种子
torch.manual_seed(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)

print("参数设置完成！")
print(f"设备: {args.device}")
print(f"模型: {args.model_name}")
print(f"Meta学习率: {args.meta_lr}")
print(f"Meta迭代次数: {args.n_meta_iterations}")
print(f"Support Set策略: 使用所有1-level difference的1D样本")
print(f"Query Set大小: {args.n_query}")

## 3. 创建模型

In [ ]:
# 创建meta-learning参数
meta_args = create_meta_learning_args(args)

# 创建模型
model = get_model(meta_args)
model = model.to(meta_args.device)

print(f"✓ 模型创建完成")
print(f"  模型类型: {type(model).__name__}")
print(f"  设备: {meta_args.device}")

## 4. Meta-Training（训练）

训练RNN成为"快速学习器"：
- 每个任务包含：Support Set（所有1-level difference的1D样本）+ Query Set（2D样本）
- 模型通过hidden state适应Support Set，然后在Query Set上测试
- 根据Query Set的表现更新模型权重

In [ ]:
# 执行meta-training
meta_trained_model, meta_losses = meta_train(
    model,
    meta_args,
    n_meta_iterations=meta_args.n_meta_iterations,
    n_tasks_per_batch=meta_args.n_tasks_per_batch
)

## 5. 训练损失可视化

In [ ]:
# 可视化训练损失
plt.figure(figsize=(12, 4))

# 原始损失
plt.subplot(1, 2, 1)
plt.plot(meta_losses, alpha=0.6, linewidth=1)
plt.xlabel('Meta-Iteration')
plt.ylabel('Meta-Loss')
plt.title('Meta-Training Loss (Raw)')
plt.grid(True, alpha=0.3)

# 平滑损失
plt.subplot(1, 2, 2)
window = min(100, len(meta_losses) // 10)
if window > 1:
    moving_avg = np.convolve(meta_losses, np.ones(window)/window, mode='valid')
    plt.plot(meta_losses, alpha=0.3, label='Raw', linewidth=1)
    plt.plot(range(window-1, len(meta_losses)), moving_avg, 
             label=f'Moving Average (window={window})', linewidth=2)
else:
    plt.plot(meta_losses, alpha=0.6, linewidth=1)
plt.xlabel('Meta-Iteration')
plt.ylabel('Meta-Loss')
plt.title('Meta-Training Loss (Smoothed)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"初始Meta-Loss: {meta_losses[0]:.4f}")
print(f"最终Meta-Loss: {meta_losses[-1]:.4f}")
if len(meta_losses) > 100:
    print(f"最后100次平均: {np.mean(meta_losses[-100:]):.4f}")

## 6. Meta-Testing（测试）

测试模型的1D→2D泛化能力：
- 冻结模型权重
- 对每个测试任务：用Support Set适应，在Query Set上测试
- 计算平均准确率

In [ ]:
# 生成测试用的grid数据
grid = GridDataGenerator(
    training_regime=args.training_regime,
    size=args.grid_size,
    use_images=args.use_images,
    image_dir=args.image_dir,
    inner_4x4=args.inner_4x4
)

# 执行meta-testing（简化版，直接测试1D→2D泛化）
final_acc, accuracies = meta_test_simple(
    meta_trained_model,
    meta_args,
    grid,
    n_test_tasks=20  # 测试20个任务
)

## 7. 结果分析和可视化

In [ ]:
# 结果总结
print("="*60)
print("📊 Meta-Testing 结果总结")
print("="*60)
print(f"\n最终平均准确率: {final_acc:.4f} ({final_acc*100:.2f}%)")
print(f"测试任务数: {len(accuracies)}")
print(f"准确率范围: {min(accuracies):.4f} - {max(accuracies):.4f}")
print(f"\n基线比较:")
print(f"  随机猜测: 50.00% (2选1)")
print(f"  人类基线: ~66-80%")

# 可视化
plt.figure(figsize=(12, 4))

# 准确率分布
plt.subplot(1, 2, 1)
plt.hist(accuracies, bins=10, alpha=0.7, edgecolor='black')
plt.axvline(final_acc, color='r', linestyle='--', linewidth=2, label=f'平均: {final_acc:.3f}')
plt.axvline(0.50, color='gray', linestyle='--', alpha=0.5, label='随机猜测 (50%)')
plt.axvline(0.66, color='green', linestyle='--', alpha=0.5, label='人类基线 (66%)')
plt.xlabel('准确率')
plt.ylabel('任务数量')
plt.title('Meta-Testing 准确率分布')
plt.legend()
plt.grid(True, alpha=0.3)

# 准确率趋势
plt.subplot(1, 2, 2)
plt.plot(range(1, len(accuracies)+1), accuracies, 'o-', alpha=0.6, markersize=4)
plt.axhline(final_acc, color='r', linestyle='--', linewidth=2, label=f'平均: {final_acc:.3f}')
plt.axhline(0.50, color='gray', linestyle='--', alpha=0.5, label='随机猜测')
plt.axhline(0.66, color='green', linestyle='--', alpha=0.5, label='人类基线')
plt.xlabel('任务序号')
plt.ylabel('准确率')
plt.title('Meta-Testing 准确率趋势')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 评估结果
print(f"\n评估:")
if final_acc > 0.66:
    print(f"  ✓ Meta-Learning成功！准确率({final_acc*100:.2f}%)超过人类基线下限")
elif final_acc > 0.50:
    print(f"  ⚠ Meta-Learning部分成功，准确率({final_acc*100:.2f}%)超过随机但未达到人类基线")
else:
    print(f"  ✗ Meta-Learning需要进一步优化，准确率({final_acc*100:.2f}%)接近随机水平")

## 8. Support Set策略说明

**当前策略**：使用所有1-level difference的1D样本

- **1-level difference**：在当前context下，两个wine的rank差值为1（相邻pair）
- **优势**：
  - 系统化学习所有相邻关系
  - 不遗漏任何重要的1D规则
  - 为2D组合提供坚实基础

- **示例**：
  - Context 0: wine1=(2,1), wine2=(1,1) → rank diff = |2-1| = 1 ✓
  - Context 0: wine1=(3,2), wine2=(1,2) → rank diff = |3-1| = 2 ✗
  - Context 1: wine1=(1,3), wine2=(1,2) → rank diff = |3-2| = 1 ✓